In [102]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

# import decision tree classifier
from sklearn.tree import DecisionTreeClassifier




In [104]:
data = pd.read_csv('output_variables_1.csv')
data
X = data.drop('Price', axis=1)
y = data[['Price']]

# do fitting
reg = LinearRegression().fit(X, y)
reg.score(X, y)
# save reg to pickle
import pickle
with open('reg.pkl', 'wb') as f:
    pickle.dump(reg, f)
    

# open reg from pickle
with open('reg.pkl', 'rb') as f:
    reg = pickle.load(f)
    
# reg predict first row
reg.predict(X.iloc[0:1, :])[0][0]
# show the coefficients and create dataframe with each of the feature name as header
coefficients = reg.coef_
coefficients = coefficients[0]
coefficients = pd.DataFrame(coefficients, index=X.columns, columns=['Coefficient'])
coefficients

,Coefficient
Net Sales,-0.178422
COGS,0.217548
Trading Income,0.000000
Other Income,-0.003289
Interest Revenue of Bank,-0.002057
Advertising & Promotion Expense,0.003691
Operational Expense,0.004847
Other Expense,-0.362341
Depreciation of Equipment Clinic Expense,-0.063470
Depreciation of Equipment Non Clinic Expense,0.000000


In [91]:
data = pd.read_csv('output_variables_1.csv')
data

,Net Sales,COGS,Trading Income,Other Income,Interest Revenue of Bank,Advertising & Promotion Expense,Operational Expense,Other Expense,Depreciation of Equipment Clinic Expense,Depreciation of Equipment Non Clinic Expense,Bank Tax Expense,Other Tax,Tangible Assets (PP&E),Net Cash Flow Average,Net Cash Flow Standard Deviation,Net Cash Flow Trend Coefficient,Equipment Usage Ratio,Total Equipments,Price
0,1862463.95,-1053087.000,0.0,6932.24,4334.5,-8064.0,-10216.0,-311702.0,0.0000,0.0,-1547.0,0.00,200000.0,68168.166667,11140.327699,-2260.559441,0.5,8,5573000
1,1486403.39,-594561.356,0.0,0.00,0.0,-285.0,0.0,-1075407.5,-133776.3051,0.0,0.0,-76677.75,200000.0,76198.416667,8229.738651,482.793706,0.5,8,6028000


In [105]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
import pickle

# Load the data
data = pd.read_csv('output_variables_1.csv')
X = data.drop('Price', axis=1).values  # Convert to NumPy array for nnls
y = data['Price'].values  # Convert to 1D NumPy array

# Perform Non-Negative Least Squares fitting
coefficients, _ = nnls(X, y)

# Create a DataFrame with feature names and coefficients
coefficients_df = pd.DataFrame(coefficients, index=data.drop('Price', axis=1).columns, columns=['Coefficient'])

# Save the coefficients to a pickle file
with open('nnls_coefficients.pkl', 'wb') as f:
    pickle.dump(coefficients_df, f)

# Load the coefficients from the pickle file
with open('nnls_coefficients.pkl', 'rb') as f:
    coefficients_df = pickle.load(f)

# Predict the first row using the fitted NNLS coefficients
first_row_prediction = np.dot(X[0, :], coefficients)

# Display the prediction for the first row and the coefficients DataFrame
print(f'Prediction for the first row: {first_row_prediction}')
print(coefficients_df)


Prediction for the first row: 5572999.999999995
                                              Coefficient
Net Sales                                        6.626121
COGS                                             6.426735
Trading Income                                   0.000000
Other Income                                     0.000000
Interest Revenue of Bank                         0.000000
Advertising & Promotion Expense                  0.000000
Operational Expense                              0.000000
Other Expense                                    0.000000
Depreciation of Equipment Clinic Expense         0.000000
Depreciation of Equipment Non Clinic Expense     0.000000
Bank Tax Expense                                 0.000000
Other Tax                                        0.000000
Tangible Assets (PP&E)                           0.000000
Net Cash Flow Average                            0.000000
Net Cash Flow Standard Deviation                 0.000000
Net Cash Flow Trend Coef

In [107]:
import pandas as pd
import numpy as np
import cvxpy as cp
import pickle

# Load the data
data = pd.read_csv('output_variables_1.csv')
X = data.drop('Price', axis=1).values  # Convert to NumPy array for cvxpy
y = data['Price'].values  # Convert to 1D NumPy array

# Define the coefficients and intercept as variables
beta = cp.Variable(X.shape[1])
intercept = cp.Variable()

# Define the objective function (minimize the least squares error)
objective = cp.Minimize(cp.sum_squares(X @ beta + intercept - y))

# Define constraints: ensure all coefficients are non-negative
constraints = [beta >= 0]

# Create and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Extract the coefficients and intercept
coefficients = beta.value
intercept_value = intercept.value

# Create a DataFrame with feature names and coefficients
coefficients_df = pd.DataFrame(coefficients, index=data.drop('Price', axis=1).columns, columns=['Coefficient'])
coefficients_df['Intercept'] = intercept_value

# Save the coefficients to a pickle file
with open('cvxpy_coefficients_non_negative.pkl', 'wb') as f:
    pickle.dump(coefficients_df, f)

# Load the coefficients from the pickle file (for testing purposes)
with open('cvxpy_coefficients_non_negative.pkl', 'rb') as f:
    coefficients_df = pickle.load(f)

# Display the coefficients DataFrame
print(coefficients_df)


                                              Coefficient     Intercept
Net Sales                                        0.027496  6.867645e+06
COGS                                             1.063607  6.867645e+06
Trading Income                                   0.000000  6.867645e+06
Other Income                                     1.322544  6.867645e+06
Interest Revenue of Bank                         1.322544  6.867645e+06
Advertising & Promotion Expense                 13.111318  6.867645e+06
Operational Expense                             13.737705  6.867645e+06
Other Expense                                    0.117528  6.867645e+06
Depreciation of Equipment Clinic Expense         1.022396  6.867645e+06
Depreciation of Equipment Non Clinic Expense     0.000000  6.867645e+06
Bank Tax Expense                                13.591698  6.867645e+06
Other Tax                                        1.301627  6.867645e+06
Tangible Assets (PP&E)                           0.000801  6.867

In [108]:
import pandas as pd
import itertools

# Define the variables
EBIT = [100000, 254786, 400000]
EBIT_Ratio = [0, 0.1, 0.22, 0.3, 0.4]
Net_Sales_Growth = [0, 0.05, 0.1, 0.2, 0.3]

# Generate all combinations
combinations = list(itertools.product(EBIT, EBIT_Ratio, Net_Sales_Growth))

# Create a DataFrame from the combinations
df = pd.DataFrame(combinations, columns=['EBIT', 'EBIT Ratio', 'Net Sales Growth'])

df.to_csv('combinations.csv', index=False)

In [118]:
data = pd.read_csv('EBIT_baseline_to_multiple.csv')

In [119]:


# Define the what-if function using the loaded data
def what_if_ebit(ebit, ebit_ratio, net_sales_growth):
    # Define boundary values based on the unique values in the dataset
    ebit_boundaries = sorted(data['EBIT'].unique())
    ebit_ratio_boundaries = sorted(data['EBIT Ratio'].unique())
    net_sales_growth_boundaries = sorted(data['Net Sales Growth'].unique())

    # Round down to nearest boundary value
    ebit = max([val for val in ebit_boundaries if val <= ebit])
    ebit_ratio = max([val for val in ebit_ratio_boundaries if val <= ebit_ratio])
    net_sales_growth = max([val for val in net_sales_growth_boundaries if val <= net_sales_growth])

    # Lookup the row with the closest match
    result = data[(data['EBIT'] == ebit) & 
                  (data['EBIT Ratio'] == ebit_ratio) & 
                  (data['Net Sales Growth'] == net_sales_growth)]

    # Return the EBIT Multiple if a match is found, else return None
    if not result.empty:
        return result['EBIT Multiple'].values[0]
    else:
        return None

# Example usage of the function with user-specified input
example_ebit = 200000
example_ebit_ratio = 0.15
example_net_sales_growth = 0.1

what_if_result = what_if_ebit(example_ebit, example_ebit_ratio, example_net_sales_growth)
what_if_result


1.05

In [120]:
# Example usage of the function
example_ebit = 400000
example_ebit_ratio = 0.05
example_net_sales_growth = 0.5

what_if_result = what_if_ebit(example_ebit, example_ebit_ratio, example_net_sales_growth)
what_if_result

4.95

In [2]:
import pandas as pd

data_df = pd.DataFrame(
    {
        "Equipment": [
        "Intra Oral Camera",
        "Bleaching Unit",
        "Ultrasonic Scaler",
        "Light Cure",
        "Dental Unit",
        "Portable Xrays",
        "Endomotor",
        "Autoclaves",
        "Ultrasonic Cleaner",
        "Water Tank",
        "Prophylaxis Hand Piece",
        "Handpiece Set",
        "Compressor",
        "Apex Locator",
        "Dental Loupe",
        "Portable Light",
        "Camera DSLR",
        "Water Tank Hose",
        "Sealing Machine",
        "Xray Sensor"
    ],
        "Own?": [True] * 20,  # Default value for ownership is True
        "Quantity": [1] * 20 ,  # Random integer values between 1 and 4,
        "Expected Lifetime": [4] * 20,  # Random integer values between 3 and 7
        "Current Lifetime Usage": [2] * 20  # Initialize with zeros
    }
)

In [6]:
# add columns to the dataframe called price
data_df['Price'] = [691.8,
195,
539.8,
238,
12200,
3900,
399,
4600,
1000,
360,
70,
800,
1330,
240,
100,
100,
1163,
4,
600,
3800]

data_df.to_csv('equipment_data.csv', index=False)

In [ ]:
import pandas as pd
import itertools

# Define the variables
Number_of_Dentist = [1, 2, 3, 4, 5]
EBIT_Ratio = [0, 0.1, 0.22, 0.3, 0.4]
Net_Sales_Growth = [0, 0.05, 0.1, 0.2, 0.3]

# Generate all combinations
combinations = list(itertools.product(EBIT, EBIT_Ratio, Net_Sales_Growth))

# Create a DataFrame from the combinations
df = pd.DataFrame(combinations, columns=['EBIT', 'EBIT Ratio', 'Net Sales Growth'])

df.to_csv('combinations_dentist_count.csv', index=False)